## In this tutorial, we'll use three different English translations of the same work, Homer's Illiad, and train a model to identify the translator given a single line of text.

In [ ]:
%pip install tensorflow-gpu

In [ ]:
import tensorflow as tf 
import os
import tensorflow_datasets as tfds

In [ ]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

In [ ]:
for name in FILE_NAMES:
  text_dir=tf.keras.utils.get_file(name,DIRECTORY_URL+name)

parent_dir=os.path.dirname(text_dir)
print(parent_dir)

/root/.keras/datasets


#Load text into datasets
Iterate through the files, loading each one into its own dataset.

Each example needs to be labeled individually labeled, so use tf.data.Dataset.map to apply a labeler function to each one. This will iterate over every example in the dataset, returning (example, label) pairs.

In [ ]:
def labeler(example,label):
  return example,tf.cast(label,tf.int64)

labeled_data_sets=[]

for i,file_name in enumerate(FILE_NAMES):
  lines_dataset=tf.data.TextLineDataset(os.path.join(parent_dir,file_name))
  labeled_dataset=lines_dataset.map(lambda ex: labeler(ex,i))
  labeled_data_sets.append(labeled_dataset)

In [ ]:
print(labeled_data_sets)

[<MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>, <MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>, <MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>]


Combine these labeled datasets into a single dataset, and shuffle it.

In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [ ]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [ ]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'The needful strength, or, scatheless yet, withdraw;'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'From his detested throat, but all around'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'occasion. Iris appears to Achilles by command of Juno, and orders him'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Companions of chill fear, from heaven infused,'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'pain and grant me strength both to cheer on the Lycians and to fight'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)


#Encode text lines as numbers
Machine learning models work on numbers, not words, so the string values need to be converted into lists of numbers. To do that, map each unique word to a unique integer.

#Build vocabulary
First, build a vocabulary by tokenizing the text into a collection of individual unique words. There are a few ways to do this in both TensorFlow and Python. For this tutorial:

Iterate over each example's numpy value.
Use tfds.features.text.Tokenizer to split it into tokens.
Collect these tokens into a Python set, to remove duplicates.
Get the size of the vocabulary for later use.

In [ ]:
tokenizer=tfds.features.text.Tokenizer()

vocabulary_set=set()

for text_tensor,_ in all_labeled_data:
  some_tokens=tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

In [ ]:
print(vocabulary_set)
print("Length of vocabulary set",len(vocabulary_set))

{'High', 'ditty', 'shortly', 'vaunted', 'tormented', 'Content', 'infused', 'hidden', 'boasts', 'exchanged', 'adventure', 'charging', 'Caeneus', 'cheerful', 'gatemen', 'attempted', 'unharmed', 'deny', 'sheaves', 'blankets', 'comes', 'Cynus', 'flitted', 'Titans', 'veriest', 'Wise', 'O', 'wounded', 'reconciles', 'juice', 'feeds', 'Sorry', 'caparison', 'Epistrophus', 'specially', 'Trachis', 'treads', 'Aesopus', 'marks', 'bowstring', 'Youthful', 'screen', 'bearing', 'leads', 'grind', 'ago', 'sav', 'hearth', 'Deceived', 'western', 'famous', 'potentate', 'children', 'snapping', 'happening', 'inscribed', 'kindle', 'victim', 'respect', 'unmeasur', 'galingal', 'Whirl', 'charms', 'Ships', 'Erichthonius', 'Appendant', 'Labors', 'Detain', 'AEsuetes', 'requite', 'court', 'marshy', 'trophies', 'wav', 'Enyeus', 'begged', 'Pydites', 'realms', 'bestowed', 'louring', 'gleanings', 'hereby', 'Wake', 'peal', 'aspect', 'Hyppotion', 'Burnt', 'Hypsipyle', 'furbish', 'List', 'kindliest', 'comforted', 'equality'

# Encode vocabulary 
We will perform encoding on the vocabulary set

In [ ]:
encoder=tfds.features.text.TokenTextEncoder(vocabulary_set)

In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = all_labeled_data.map(encode_map_fn)

Split dataset into training and testing dataset

In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

Now we calculate the vocabulary size and add 1 to it since we added padding to text

In [ ]:
vocab_size=len(vocabulary_set)+1

Building the model of dataset

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 64))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [ ]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Now training and validating dataset with the model.

In [ ]:
model.fit(train_data,epochs=10,validation_data=test_data)

Epoch 1/10
697/697 [==============================] - 20s 29ms/step - loss: 0.5214 - accuracy: 0.7406 - val_loss: 0.3946 - val_accuracy: 0.8200
Epoch 2/10
697/697 [==============================] - 19s 27ms/step - loss: 0.3008 - accuracy: 0.8683 - val_loss: 0.3766 - val_accuracy: 0.8210
Epoch 3/10
697/697 [==============================] - 19s 27ms/step - loss: 0.2247 - accuracy: 0.9037 - val_loss: 0.3781 - val_accuracy: 0.8376
Epoch 4/10
697/697 [==============================] - 19s 27ms/step - loss: 0.1809 - accuracy: 0.9233 - val_loss: 0.4286 - val_accuracy: 0.8392
Epoch 5/10
697/697 [==============================] - 19s 27ms/step - loss: 0.1479 - accuracy: 0.9358 - val_loss: 0.4653 - val_accuracy: 0.8416
Epoch 6/10
697/697 [==============================] - 19s 27ms/step - loss: 0.1196 - accuracy: 0.9476 - val_loss: 0.5455 - val_accuracy: 0.8356
Epoch 7/10
697/697 [==============================] - 19s 27ms/step - loss: 0.1004 - accuracy: 0.9539 - val_loss: 0.6004 - val_accuracy:

In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)
print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

79/79 [==============================] - 2s 29ms/step - loss: 0.7928 - accuracy: 0.8286

Eval loss: 0.793, Eval accuracy: 0.829
